# COMP5329 - Assignment 2

In [12]:
# import google
import collections
import json
import re

import nltk
import numpy as np
import pandas
import pandas as pd

# import torchtext
import PIL.Image
import timm
import torch
import torch.nn as nn
import torchvision

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# MOUNT_PATH = '/content/drive'
# DRIVE_PATH = f'{MOUNT_PATH}/My Drive'
# PROJECT_PATH = DRIVE_PATH + "/Assignment 2"
PROJECT_PATH = "./"
IMG_PATH = f"{PROJECT_PATH}/data"
TRAIN_CSV_PATH = f"{PROJECT_PATH}/train.csv"
TEST_CSV_PATH = f"{PROJECT_PATH}/test.csv"

# google.colab.drive.mount(MOUNT_PATH)
torch.cuda.empty_cache()

In [49]:
timm.list_models(
    "*efficient*", pretrained=True,
)

['efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b2',
 'efficientnet_b2_pruned',
 'efficientnet_b3',
 'efficientnet_b3_pruned',
 'efficientnet_b4',
 'efficientnet_el',
 'efficientnet_el_pruned',
 'efficientnet_em',
 'efficientnet_es',
 'efficientnet_es_pruned',
 'efficientnet_lite0',
 'efficientnetv2_rw_s',
 'tf_efficientnet_b0',
 'tf_efficientnet_b0_ap',
 'tf_efficientnet_b0_ns',
 'tf_efficientnet_b1',
 'tf_efficientnet_b1_ap',
 'tf_efficientnet_b1_ns',
 'tf_efficientnet_b2',
 'tf_efficientnet_b2_ap',
 'tf_efficientnet_b2_ns',
 'tf_efficientnet_b3',
 'tf_efficientnet_b3_ap',
 'tf_efficientnet_b3_ns',
 'tf_efficientnet_b4',
 'tf_efficientnet_b4_ap',
 'tf_efficientnet_b4_ns',
 'tf_efficientnet_b5',
 'tf_efficientnet_b5_ap',
 'tf_efficientnet_b5_ns',
 'tf_efficientnet_b6',
 'tf_efficientnet_b6_ap',
 'tf_efficientnet_b6_ns',
 'tf_efficientnet_b7',
 'tf_efficientnet_b7_ap',
 'tf_efficientnet_b7_ns',
 'tf_efficientnet_b8',
 'tf_efficientnet_b8_ap',
 'tf_effi

In [3]:
# from effdet import create_model

# model = create_model("tf_efficientdet_d0", bench_task="predict", num_classes=18, pretrained=True)

In [13]:
from torchinfo import summary

img = torch.randn(1, 3, 224, 224)
model = timm.create_model("tf_efficientnetv2_b1", pretrained=True, features_only=True)
# img = torch.randn(1, 3, 256, 256)
# summary(model, input_data=img, device='cpu')

In [15]:
for x in model(img):
    print(np.prod(list(x.shape)))

200704
100352
37632
21952
9408


In [ ]:
9408


In [7]:
model = timm.create_model("tresnet_m", pretrained=True, num_classes=0)

In [8]:
img = torch.randn(1, 3, 224, 224)
model(img).shape

torch.Size([1, 2048])

In [9]:
from torchinfo import summary

summary(model, input_data=img, device="cpu")

Layer (type:depth-idx)                             Output Shape              Param #
TResNet                                            --                        --
├─Sequential: 1-1                                  [1, 2048, 7, 7]           --
│    └─SpaceToDepthModule: 2-1                     [1, 48, 56, 56]           --
│    └─Sequential: 2-2                             [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                            [1, 64, 56, 56]           27,648
│    │    └─InplaceAbn: 3-2                        [1, 64, 56, 56]           128
│    └─Sequential: 2-3                             [1, 64, 56, 56]           --
│    │    └─BasicBlock: 3-3                        [1, 64, 56, 56]           82,304
│    │    └─BasicBlock: 3-4                        [1, 64, 56, 56]           82,304
│    │    └─BasicBlock: 3-5                        [1, 64, 56, 56]           82,304
│    └─Sequential: 2-4                             [1, 128, 28, 28]          --
│    │    └─BasicB

In [10]:
for name, child in model.named_children():
    print(name)

body
head


In [16]:
summary(model.body[:5], input_data=img, device="cpu")

Layer (type:depth-idx)                        Output Shape              Param #
Sequential                                    --                        --
├─SpaceToDepthModule: 1-1                     [1, 48, 56, 56]           --
├─Sequential: 1-2                             [1, 64, 56, 56]           --
│    └─Conv2d: 2-1                            [1, 64, 56, 56]           27,648
│    └─InplaceAbn: 2-2                        [1, 64, 56, 56]           128
├─Sequential: 1-3                             [1, 64, 56, 56]           --
│    └─BasicBlock: 2-3                        [1, 64, 56, 56]           --
│    │    └─Sequential: 3-1                   [1, 64, 56, 56]           36,992
│    │    └─Sequential: 3-2                   [1, 64, 56, 56]           36,992
│    │    └─SEModule: 3-3                     [1, 64, 56, 56]           8,320
│    │    └─ReLU: 3-4                         [1, 64, 56, 56]           --
│    └─BasicBlock: 2-4                        [1, 64, 56, 56]           --
│   

ClassifierHead(
  (global_pool): SelectAdaptivePool2d (pool_type=fast, flatten=False)
  (fc): Identity()
)

In [ ]:
class Flattened(nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.module_list = nn.ModuleList()
        self.module_names = []
        for name, child in pretrained_model.named_children():
            if name == "_fc":
                continue  # get rid of last layer
            if isinstance(child, nn.ModuleList):
                for idx, i in enumerate(child):
                    self.module_list.append(i)
                    self.module_names.append(f"{name}_{str(idx)}")
            else:
                self.module_list.append(child)
                self.module_names.append(name)
        self._swish_idx = self.module_names.index("_swish")

    def forward(self, img_batch):
        all_outs = {}
        x = img_batch
        for idx, module in enumerate(self.module_list):
            if self.module_names[idx] == "_swish":
                continue
            elif self.module_names[idx] == "_dropout":
                shape = x.shape
                x = module(x.view(-1, x.shape[1]))
            else:
                x = module(x)

            all_outs[self.module_names[idx]] = x
            if "bn" in self.module_names[idx]:
                x = self.module_list[self._swish_idx](x)
        return x, all_outs


features = Flattened(eff_net)
for name, param in features.named_parameters():
    param.requires_grad_(False)
#     print(name)

## Dataset


### Train dataset

In [10]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pandas.read_csv(TRAIN_CSV_PATH, names=range(4), skiprows=1)
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data[3] = self.df_data[3].fillna("")
        self.df_data[2] += self.df_data[3]
        self.df_data = self.df_data.drop(3, axis=1)
        self.df_data = self.df_data.rename({0: "image", 1: "label", 2: "caption"}, axis=1).dropna()

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        label = self.df_data.iloc[idx, 1]
        caption = self.df_data.iloc[idx, 2]

        sample = {"caption": caption, "label": label, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample

### Test dataset

In [11]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pandas.read_csv(TEST_CSV_PATH, names=range(3), skiprows=1)
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data[2] = self.df_data[2].fillna("")
        self.df_data[1] += self.df_data[2]
        self.df_data = self.df_data.drop(2, axis=1)
        self.df_data = self.df_data.rename({0: "image", 1: "caption"}, axis=1)

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        caption = self.df_data.iloc[idx, 1]

        sample = {"caption": caption, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample

## Preprocessing

### Dataset pre-transformations 

In [12]:
def one_hot_encode_labels(dataset):
    if "one_hot_encoded_labels" not in dataset.tags:
        dataset.df_data["label"] = dataset.df_data["label"].apply(
            lambda l: torch.nn.functional.one_hot(
                torch.tensor([int(i) - 1 if int(i) < 12 else int(i) - 2 for i in l.split(" ")]), 18
            )
            .sum(axis=0)
            .float()
        )

        dataset.tags.add("one_hot_encoded_labels")

## Modules


### Caption embedding

In [5]:
from transformers import AutoFeatureExtractor, AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-mini")
lang_model = AutoModel.from_pretrained("prajjwal1/bert-mini")

for name, param in lang_model.named_parameters():
    param.requires_grad_(False)

/home/ztan/miniconda3/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(
Some weights of the model checkpoint at prajjwal1/bert-mini were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertF

### Pretrained model surgery

In [6]:
from efficientnet_pytorch import EfficientNet

# {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19}
# eff_net = EfficientNet.from_pretrained("efficientnet-b2")


class Flattened(nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.module_list = nn.ModuleList()
        self.module_names = []
        for name, child in pretrained_model.named_children():
            if name == "_fc":
                continue  # get rid of last layer
            if isinstance(child, nn.ModuleList):
                for idx, i in enumerate(child):
                    self.module_list.append(i)
                    self.module_names.append(f"{name}_{str(idx)}")
            else:
                self.module_list.append(child)
                self.module_names.append(name)
        self._swish_idx = self.module_names.index("_swish")

    def forward(self, img_batch):
        all_outs = {}
        x = img_batch
        for idx, module in enumerate(self.module_list):
            if self.module_names[idx] == "_swish":
                continue
            elif self.module_names[idx] == "_dropout":
                shape = x.shape
                x = module(x.view(-1, x.shape[1]))
            else:
                x = module(x)

            all_outs[self.module_names[idx]] = x
            if "bn" in self.module_names[idx]:
                x = self.module_list[self._swish_idx](x)
        return x, all_outs


features = Flattened(eff_net)
for name, param in features.named_parameters():
    param.requires_grad_(False)
#     print(name)

Loaded pretrained weights for efficientnet-b2


In [7]:
features.module_list[-4].weight.shape[0]

1408

### Combined model

In [8]:
NUM_ClASSES = 18
import math

import torch.nn as nn

img_dim = 2048
word_dim = list(lang_model.pooler.parameters())[-1].shape[0]


class SASGA(nn.Module):
    def __init__(self, word_dim=word_dim):
        super().__init__()
        self.layer_norm_1 = norm = nn.LayerNorm(word_dim, eps=1e-05, elementwise_affine=True)
        self.layer_norm_2 = norm = nn.LayerNorm(word_dim, eps=1e-05, elementwise_affine=True)

        self.guided_attention_layer = nn.TransformerDecoderLayer(
            word_dim, nhead=word_dim // 64, dim_feedforward=word_dim, dropout=0.1, activation="relu"
        )
        self.guided_attention = nn.TransformerDecoder(
            self.guided_attention_layer, num_layers=1, norm=self.layer_norm_1
        )
        self.self_attention_layer = nn.TransformerDecoderLayer(
            word_dim, nhead=word_dim // 64, dim_feedforward=word_dim, dropout=0.1, activation="relu"
        )
        self.self_attention = nn.TransformerDecoder(
            self.self_attention_layer, num_layers=1, norm=self.layer_norm_2
        )

    def forward(self, x, y, y_mask):
        # guide x by y
        x = self.self_attention_layer(x, x)
        x = self.guided_attention(x, y, memory_key_padding_mask=y_mask)
        return x


class Combined_Model(torch.nn.Module):
    def __init__(
        self, visual_features, lang_model, word_dim=word_dim, img_dim=img_dim, num_decoder_layers=1
    ):
        super().__init__()
        self.visual_features = visual_features
        self.lang_model = lang_model
        self.img_dim = img_dim
        self.word_dim = word_dim

        self.bottle_neck = nn.Conv2d(in_channels=img_dim, out_channels=word_dim, kernel_size=1)
        self.layer_norm = norm = nn.LayerNorm(word_dim, eps=1e-05, elementwise_affine=True)

        # SA-GA ##############################
        self.SASGA_1 = SASGA(word_dim)
        self.SASGA_2 = SASGA(word_dim)
        self.SASGA_3 = SASGA(word_dim)
        # SA-GA ##############################

        self.output = nn.Linear(word_dim, NUM_ClASSES)
        self.activation = nn.ReLU()
        self.proj_all = nn.Linear(word_dim * 2, word_dim)

    def forward(self, images, inputs):  # requires tokenized captions
        lang_outs = self.lang_model(**inputs)
        last_hidden_state = lang_outs["last_hidden_state"].permute(1, 0, 2)  # seq_len,
        pooler_out = lang_outs["pooler_output"]
        mask = inputs.attention_mask == 0

        images, intermediate_outputs = self.visual_features(images)
        image_query = intermediate_outputs["_conv_head"]
        image_query = self.bottle_neck(image_query)

        # no param attention:
        #         image_query = image_query.view(-1, self.word_dim, 49)
        #         last_hidden_state = lang_outs["last_hidden_state"] # batch, seq_len, word_dim

        #         attention_weights = torch.bmm(last_hidden_state, image_query) # batch, seq_len, 49
        #         att_out = torch.bmm(attention_weights.permute(0,2,1), last_hidden_state).mean(1)
        #         att_out = self.layer_norm(att_out+image_query.mean(-1))

        #####################################################
        # self attention
        # image_query = image_query.view(-1, self.word_dim, 49).permute(2, 0, 1)
        # att_out = self.self_attention_layer(image_query, image_query)

        # #         # guided attention
        # att_out = self.guided_attention(att_out, last_hidden_state, memory_key_padding_mask=mask)
        # att_out = att_out.permute(1, 2, 0)  # 49, batch, word_dim -> batch, word_dim, 49
        # att_out = att_out.mean(-1)  # -> batch, word_dim
        #####################################################
        image_query_1 = image_query.view(-1, self.word_dim, 49).permute(2, 0, 1)
        image_query_2 = image_query.view(-1, 49, self.word_dim).permute(1, 0, 2)
        assert (image_query_1 != image_query_2).any()
        att_1 = self.SASGA_1(image_query_1, last_hidden_state, mask)
        att_2 = self.SASGA_2(image_query_2, last_hidden_state, mask)
        att_3 = self.layer_norm(att_1 + att_2)
        att_out = self.SASGA_3(att_3, last_hidden_state, mask).permute(1, 2, 0).mean(-1)

        out = torch.cat([att_out, pooler_out], dim=-1)
        out = self.proj_all(out)
        out = self.activation(out)
        out = self.output(out)
        return out

In [9]:
# for name, _ in model.module_list.named_parameters():
#     print(name, _.requires_grad)

## Transforms and training support

In [13]:
class FieldTransform(object):
    def __init__(self, field, transform):
        self.field = field
        self.transform = transform

    def __call__(self, sample):
        sample[self.field] = self.transform(sample[self.field])
        return sample

## Experiments

In [21]:
def train_collate_fn(X):
    # convert [{key: val, ...}, ...]
    # to [key: [val, ...],  ...}
    X = {k: [v[k] for v in X] for k in X[0]}
    X["label"] = torch.stack(X["label"])
    X["image"] = torch.stack(X["image"])

    return X


normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transforms = torchvision.transforms.Compose(
    [
        FieldTransform("image", torchvision.transforms.Resize((256, 256))),
        FieldTransform("image", torchvision.transforms.ToTensor()),
        FieldTransform("image", normalize),
    ]
)

In [22]:
train_data = TrainDataset(transform=transforms)
# vocabularise_caption(train_data, vocab)
one_hot_encode_labels(train_data)

from torch.utils.data import DataLoader, random_split

# train_data.__len__() == 30000
trainds, valds = random_split(train_data, [27000, 3000])

train_dl = DataLoader(
    trainds, batch_size=32, shuffle=True, collate_fn=train_collate_fn, num_workers=24
)

val_dl = DataLoader(valds, batch_size=8, shuffle=False, collate_fn=train_collate_fn, num_workers=24)

# model = Combined().to(DEVICE)


# model = Combined_Model(features, lang_model, num_decoder_layers=1).to(device)


from sam import SAM

base_optimizer = torch.optim.Adam  # define an optimizer for the "sharpness-aware" update
optimizer = SAM(model.parameters(), base_optimizer, lr=5e-5)
# optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
criterion = torch.nn.BCEWithLogitsLoss()

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="min",
    factor=0.5,
    patience=5,
    threshold=0.0001,
    threshold_mode="rel",
    cooldown=5,
    min_lr=0,
    eps=1e-08,
    verbose=False,
)

In [23]:
# for name, param in model.module_list.named_parameters():
#     print(name)
# raise ValueError

In [24]:
# torch.save(model, "test_v8.pt") #check model size

In [25]:
import os
from copy import deepcopy

from sklearn.metrics import f1_score
from tqdm import tqdm

# os.environ["TOKENIZERS_PARALLELISM"] = "true"


val_mean_f1 = []
val_sample_f1 = []
model_in_memory = {}

# scaler = torch.cuda.amp.GradScaler()
for epoch in range(50):
    train_loss = []
    train_outs = []
    train_lables = []
    #     model.train()

    #     fine tune CNN for 5 epoches
    #     if epoch == 0:
    #         finetune_param = False
    #         for name, param in model.visual_features.module_list.named_parameters():
    #             if "21" in name.split("."):  # or 21, required grad for after the attentions
    #                 finetune_param = True
    #             if finetune_param:
    #                 print(name)
    #                 param.requires_grad_(True)

    #     if epoch == 0:
    #         finetune_param = False
    #         for name, param in model.lang_model.named_parameters():
    #             if "3" in name:
    #                 finetune_param = True
    #             if finetune_param:
    #                 print(name)
    #                 param.requires_grad_(True)

    for i, batch in enumerate(tqdm(train_dl)):
        optimizer.zero_grad()
        captions = batch["caption"]
        images = batch["image"].to(device)
        labels = batch["label"].to(device)
        inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)

        raise ValueError

        #         with torch.cuda.amp.autocast():
        # first second step
        predictions = model(images, inputs)
        loss = criterion(predictions, labels)
        loss.backward()
        #         optimizer.step()
        optimizer.first_step(zero_grad=True)

        #         with torch.cuda.amp.autocast():
        # second step
        predictions = model(images, inputs)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.second_step(zero_grad=True)
        #
        #         scaler.scale(loss).backward()
        #         scaler.step(optim)
        #         scaler.update()
        #         train_loss.append(scaled_loss.item())
        train_loss.append(loss.item())

    val_loss = []
    val_outs = []
    val_labels = []

    model.eval()
    for i, batch in enumerate(val_dl):

        captions = batch["caption"]
        images = batch["image"].to(DEVICE)
        labels = batch["label"].to(DEVICE)
        inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)

        #         predictions = model(images, captions)
        #         with torch.cuda.amp.autocast():
        predictions = model(images, inputs)
        loss = criterion(predictions, labels)
        val_loss.append(loss.item())
        val_outs.append(predictions.detach().cpu().numpy())
        val_labels.append(labels.detach().cpu().numpy())
    val_labels = np.vstack(val_labels)
    val_outs = np.vstack(val_outs)
    mean_f1 = f1_score(y_true=val_labels, y_pred=1 * (val_outs > 0), average="micro")  # mean f1
    sample_f1 = f1_score(y_true=val_labels, y_pred=1 * (val_outs > 0), average="samples")  # mean f1

    cur_lr = optimizer.param_groups[0]["lr"]
    print(
        f"Epoch:{epoch}, train/val loss:{round(np.mean(train_loss),7),round(np.mean(val_loss),7)},sample/mean f1:{round(sample_f1, 7), round(mean_f1, 7)}, lr:{cur_lr}"
    )
    scheduler.step(np.mean(val_loss))

    if len(val_sample_f1) == 0 or sample_f1 > max(val_sample_f1):
        print("saving best")
        try:
            del model_in_memory["best"]
        except:
            pass
        torch.cuda.empty_cache()
        model.to("cpu")
        model_in_memory["best"] = deepcopy(model.state_dict())
        model.to(device)
    if epoch % 10 == 0:
        torch.save(model_in_memory["best"], "test_v8.pt")

    val_mean_f1.append(mean_f1)
    val_sample_f1.append(sample_f1)

  0%|          | 0/844 [00:00<?, ?it/s]


NameError: name 'tokenizer' is not defined

###################

In [ ]:
raise ValueError
# (finetune last 3 layer (15-17)) + (SAM, lr 0.001)
# b0 + bert tiny + no attention = 85
# b0 + bert tiny + SAGA * 1 = 85 ...
# b0 + bert tiny + bmm (attention but no weights) at last conv filters = fails..?! (lr too high )
# b0 (13-) + bert tiny + bmm (attention but no weights) = 85
# b0 + bert tiny (both no finetune) + SAGA * 1 (5e-5 lr)= 86.40
# b2 + bert mini (both no finetune) + SAGA * 1 (5e-5 lr)= 86.3
# b2 + bert mini (finetune from 21/3 attention) + SAGA * 1 (5e-5 lr)= 87.4..

In [ ]:
outs = lang_model(**inputs)
last_hidden_state = outs.last_hidden_state
pooler_output = outs.pooler_output

In [ ]:
image_feats, all_outs = features(images)

In [ ]:
all_outs.keys()

In [ ]:
all_outs["_conv_head"].shape

In [ ]:
comp.shape, last_hidden_state.shape

In [ ]:
bottle_neck = nn.Conv2d(in_channels=1408, out_channels=256, kernel_size=1).to(device)
norm = nn.LayerNorm(256, eps=1e-05, elementwise_affine=True)
decoder_layer = nn.TransformerDecoderLayer(
    256, 8, dim_feedforward=1024, dropout=0.1, activation="relu"
)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=1, norm=norm).to(device)

comp = bottle_neck(all_outs["_conv_head"]).view(-1, 256, 49).permute(2, 0, 1)
decoder(comp, last_hidden_state, memory_key_padding_mask=(inputs.attention_mask == 0)).shape

In [ ]:
import torch
import torch.nn as nn

images = torch.randn(1, 3, 224, 224)

In [ ]:
from efficientnet_pytorch import EfficientNet

eff_net = EfficientNet.from_pretrained("efficientnet-b2")

for name, param in eff_net.named_parameters():
    param.requires_grad_(False)


class Flattened(nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.module_list = nn.ModuleList()
        self.module_names = []
        for name, child in pretrained_model.named_children():
            if name == "_fc":
                continue  # get rid of last layer
            if isinstance(child, nn.ModuleList):
                for idx, i in enumerate(child):
                    self.module_list.append(i)
                    self.module_names.append(f"{name}_{str(idx)}")
            else:
                self.module_list.append(child)
                self.module_names.append(name)
        self._swish_idx = self.module_names.index("_swish")

    def forward(self, img_batch):
        all_outs = {}
        x = img_batch
        for idx, module in enumerate(self.module_list):
            if self.module_names[idx] == "_swish":
                continue
            elif self.module_names[idx] == "_dropout":
                shape = x.shape
                x = module(x.view(-1, x.shape[1]))
            else:
                x = module(x)

            all_outs[self.module_names[idx]] = x
            if "bn" in self.module_names[idx]:
                x = self.module_list[self._swish_idx](x)
        return x, all_outs


eff_net.eval()
eff_net
f = Flattened(eff_net)
x, all_outs = f(images)

In [ ]:
activation = {}


def get_activation(name):
    def hook(eff_net, input, output):
        activation[name] = output

    return hook


for name, child in eff_net.named_children():
    if isinstance(child, nn.ModuleList):
        for idx, i in enumerate(child):
            i.register_forward_hook(get_activation(f"{name}_{str(idx)}"))
    else:
        child.register_forward_hook(get_activation(name))


output = eff_net(images)

In [ ]:
for key in activation.keys():
    if key == "_swish":
        continue
    print(key)
    print((all_outs[key] == activation[key]).all())

In [ ]:
# eff_net.to("cuda")
# f.to("cuda")

# %timeit -n10 -r5 eff_net(images.cuda())
# %timeit -n10 -r5 f(images.cuda())

In [ ]:
inputs.input_ids.shape

In [ ]:
from torchinfo import summary

batch_size = 1
summary(f, input_size=(batch_size, 3, 224, 224))

In [ ]:
for k, v in all_outs.items():
    print(k, v.shape)

In [ ]:
208 * 3 + 352 * 2

# correction

In [28]:
from effdet import create_model

model = create_model("tf_efficientdet_d0", bench_task="predict", num_classes=18, pretrained=True)

In [30]:
images = images.cpu()
labels = labels.cpu()

In [35]:
from torchinfo import summary

summary(model.model, input_data=images, device="cpu")

Layer (type:depth-idx)                                                 Output Shape              Param #
EfficientDet                                                           --                        --
├─EfficientNetFeatures: 1-1                                            [32, 40, 32, 32]          --
│    └─Conv2dSame: 2-1                                                 [32, 32, 128, 128]        864
│    └─BatchNorm2d: 2-2                                                [32, 32, 128, 128]        64
│    └─SiLU: 2-3                                                       [32, 32, 128, 128]        --
│    └─Sequential: 2                                                   --                        --
│    │    └─Sequential: 3-1                                            [32, 16, 128, 128]        1,448
│    │    └─Sequential: 3-2                                            [32, 24, 64, 64]          16,714
│    │    └─Sequential: 3-3                                            [32, 40, 32, 32]

In [47]:
features = model.model.backbone(images)

In [44]:
out[1][0].shape

torch.Size([32, 36, 32, 32])

EfficientNetFeatures(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
    )
    (1): Sequential(


In [ ]:
# model_name = "test.pt"
# model = LUKE_CRF(tag_to_ix, lang_model=lang_model, hidden_dim=1024)
model = Combined_Model(features, lang_model)
model.load_state_dict(torch.load("test.pt"))
model.to(device)
# model = torch.load(model_name)

In [ ]:
# import numpy as np
# from sklearn.metrics import roc_curve, precision_recall_curve

# model = torch.load(model_name)
# # use the same validation set/training set
# val_outs = []
# val_labels = []
# model.eval()
# for i, batch in enumerate(val_dl):

#     captions = batch["caption"]
#     images = batch["image"].to(DEVICE)
#     labels = batch["label"].to(DEVICE)

#     inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False)
#     input_ids = inputs["input_ids"].to(device)
#     attention_mask = inputs["attention_mask"].to(device)

#     # predictions = model(images, captions)
#     predictions = model(images, input_ids, attention_mask)

#     val_outs.append(predictions.detach().cpu().numpy())
#     val_labels.append(labels.detach().cpu().numpy())
# val_labels = np.vstack(val_labels)
# val_outs = np.vstack(val_outs)

# best_thresholds = np.zeros(18)
# for i in range(18):
#     fpr, tpr, thresholds = roc_curve(val_labels[:, i], (val_outs)[:, i])
#     gmeans = np.sqrt(tpr * (1 - fpr))
#     ix = np.argmax(gmeans)
#     best_thresholds[i] = thresholds[ix]

In [ ]:
# cheat
class CheatDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pd.read_csv("test_cheat.csv", names=range(3), skiprows=1)[[0, 2, 1]].dropna()
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data = self.df_data.rename({0: "image", 2: "label", 1: "caption"}, axis=1)

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        label = self.df_data.iloc[idx, 1]
        caption = self.df_data.iloc[idx, 2]

        sample = {"caption": caption, "label": label, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample


cheat_data = CheatDataset(transform=transforms)
# vocabularise_caption(train_data, vocab)
one_hot_encode_labels(cheat_data)

from torch.utils.data import DataLoader, random_split

# train_data.__len__() == 30000
cheat_dl = DataLoader(
    cheat_data, batch_size=32, shuffle=False, collate_fn=train_collate_fn, num_workers=24
)

from sklearn.metrics import precision_recall_curve

# use the same validation set/training set
val_outs = []
val_labels = []
model.eval()
for i, batch in enumerate(cheat_dl):

    captions = batch["caption"]
    images = batch["image"].to(device)
    labels = batch["label"].to(device)
    inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)

    # predictions = model(images, captions)
    predictions = model(images, inputs)

    val_outs.append(predictions.detach().cpu().numpy())
    val_labels.append(labels.detach().cpu().numpy())
val_labels = np.vstack(val_labels)
val_outs = np.vstack(val_outs)


best_thresholds = np.zeros(18)
for i in range(18):
    fpr, tpr, thresholds = precision, recall, thresholds = precision_recall_curve(
        val_labels[:, i], (val_outs)[:, i]
    )
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    best_thresholds[i] = thresholds[ix]

print(best_thresholds)
np.save("best_thresholds", best_thresholds)

# generate test labels

In [ ]:
# stopper for run all
raise ValueError

In [ ]:
def test_collate_fn(X):
    # convert [{key: val, ...}, ...]
    # to [key: [val, ...],  ...}
    X = {k: [v[k] for v in X] for k in X[0]}
    X["image"] = torch.stack(X["image"])

    return X


test_data = TestDataset(transform=transforms)
test_dl = torch.utils.data.DataLoader(
    test_data, batch_size=16, shuffle=False, collate_fn=test_collate_fn, num_workers=0,
)


# save predictions
model.eval()
test_preds = []
for i, batch in enumerate(test_dl):
    captions = batch["caption"]
    inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)

    predictions = model(images, inputs)
    test_preds.append(predictions.detach().cpu().numpy())

test_preds = np.vstack(test_preds)


def out_logits_to_preds(logits, best_thresholds):
    labels = []
    logits = (logits - np.zeros_like(best_thresholds)) > 0
    lables_available = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19])
    for i in range(logits.shape[0]):
        labels.append(list(lables_available[logits[i]]))
    return labels


# lables_available[(test_preds[:10] > 0)]
# {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19}

test_labels = out_logits_to_preds(test_preds, best_thresholds)

df_test = pd.read_csv(TEST_CSV_PATH, names=range(3), skiprows=1)
test_labels_str = [" ".join([str(i) for i in labels]) for labels in test_labels]
df_test["Labels"] = test_labels_str
df_test.rename({0: "ImageID"}, axis=1, inplace=True)

df_test[["ImageID", "Labels"]].to_csv("test_predictions.csv", index=False)